# 1 - PROBLEM

## Requisitos

1. **Leitura de Dados:** Desenvolva um componente capaz de ler uma lista de objetos contidos em um arquivo JSON, representando informações sobre tema escolhidoes.

2. **Mapeamento, Filtro e Redução:** Implemente funcionalidades para realizar mapeamento, filtragem e redução dos dados, proporcionando uma análise mais refinada das informações contidas no conjunto de dados.

3. **Manipulação de Dados Individuais:** Permita a leitura individual, atualização e exclusão de do arquivossores, mantendo o arquivo JSON sempre atualizado.

4. **Validações de Operações:** Integre validações utilizando blocos try-except e raise para garantir a robustez das operações, prevenindo erros e assegurando a consistência dos dados.

5. **Obtenção de Estatísticas Simples:** Desenvolva uma função para extrair dados estatísticos simples, como média, máximo e mínimo, por exemplo, aum item X nos dados de exatas.

6. **Identificação de Máximos/Mínimos com Detalhes:** Crie uma função que retorne uma lista de tuplas, contendXdo professor e o valor máximo (ou mínimo) de algum atributo numérico. Esta função deve ser configurável para fornecer estatísticas de máximo ou mínimo.

7. **Exportação de Dados Estatísticos para CSV:** Implemente a capacidade de salvar os dados estatísticos obtidos em um arquivo CSV, permitindo uma análise posterior ou compartilhamento fácil dos resultados.

# 2 - IMPORTS

## 2.1 - Bibliotecas

In [1]:
import json

## 2.2 - Funções

### 2.2.1 - Leitura do Arquivo

In [2]:
def carrega_dados(path:str = 'data/receitas.json') -> list[dict]:
    try:
        with open(path, 'r', encoding='utf-8') as arquivo:
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError:
        return []

### 2.2.2 - Salvamento do Arquivo

In [3]:
def salvar_dados(dados:list[dict], path:str = 'data/receitas.json') -> bool:
    try:
        with open(path, 'w') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

### 2.2.3 - Visualização do Arquivo

In [4]:
def formata_lista_receitas(lista_receitas):
    return '\n'.join(map(formata_receita, lista_receitas))

In [5]:
def formata_receita(receita):
    nome = receita['nome']
    ingredientes = formata_ingredientes(receita['ingredientes'])
    instrucoes = receita['instrucoes']
    return f'{nome}:\n Ingredientes:\n  {ingredientes}\n Instruções: {instrucoes}\n'

In [6]:
def formata_ingredientes(ingredientes: list) -> str:
    return '\n  '.join(map(formata_dicionario, ingredientes))

In [7]:
def formata_dicionario(dados: dict) -> str:
    return ' | '.join([f'{k}: {v}' for k, v in dados.items()])

### 2.2.4 - Leitura do Arquivo

In [8]:
def retorna_dicionario(dado: dict, escolha=''):
    return dado if escolha in dado['nome'] else None

In [9]:
def verifica_escolha(data, escolha=''):
    resultado = []
    for dado in data:
        if retorna_dicionario(dado, escolha) is not None:
            resultado.append(dado)
    return resultado

# 3 - Code Start

## 3.1 - Carregamento de Dados

In [10]:
url = ''

In [11]:
dados = carrega_dados(url)

In [12]:
dados

[]

## 3.2 - Visualizar os dados carregados

In [13]:
resultado_formatado = formata_lista_receitas(dados)

In [14]:
print(resultado_formatado)

## 3.3 - Leitura Individual

In [15]:
x = verifica_escolha(dados, 'Salada Caesar')

In [16]:
x

[]